In [59]:
import pandas

In [60]:
import xml.dom.minidom

def print_xml_string(xml_string):
  dom = xml.dom.minidom.parseString(xml_string)

  # Pretty print the XML
  pretty_xml = dom.toprettyxml()

  # Print or save the pretty-printed XML
  print(pretty_xml)

In [61]:
import xml.etree.ElementTree as ET
import csv


filenames = ['Sprawozdania[2022][IVKwartał] Wydatki.xml',
         'Sprawozdania[2022][IVKwartał] Dochody.xml',
         'Sprawozdania[2023][IIKwartał] Wydatki.xml',
         'Sprawozdania[2023][IIKwartał] Dochody.xml']

In [62]:
with open(filenames[0]) as f:
  xml_string = f.read()
  print_xml_string(xml_string)

<?xml version="1.0" ?>
<PaczkaSprawozdan Wersja="0.3">
	<Okres>
		<Rok>2022</Rok>
		<TypOkresu>Kwartal</TypOkresu>
		<Okres>4</Okres>
	</Okres>
	<Jednostki>
		<Jednostka>
			<Nazwa>Szkoła Podstawowa Nr 7</Nazwa>
			<Typ>JB</Typ>
			<Regon>001070630     </Regon>
			<WK>10</WK>
			<PK>02</PK>
			<GK>01</GK>
			<GT>1</GT>
			<PT>0</PT>
			<Sprawozdania>
				<Rb-28s Id="Rb28s_216eefc41e244a948edccfe86f061350">
					<Okres>
						<Rok>2022</Rok>
						<TypOkresu>Kwartal</TypOkresu>
						<Okres>4</Okres>
					</Okres>
					<Jednostka>
						<Nazwa>Szkoła Podstawowa Nr 7</Nazwa>
						<Typ>JB</Typ>
						<Regon>001070630     </Regon>
						<WK>10</WK>
						<PK>02</PK>
						<GK>01</GK>
						<GT>1</GT>
						<PT>0</PT>
					</Jednostka>
					<Naglowek>
						<Wersja>1</Wersja>
						<DataSprawozdania>2023-02-07</DataSprawozdania>
						<DostepneGrupyParagrafow>false</DostepneGrupyParagrafow>
					</Naglowek>
					<Pozycje>
						<Pozycja>
							<Dzial>801</Dzial>
							<Rozdzial>80101</Ro

In [63]:
unique_field_names = set()

def add_fields_to_dict(obj, row, fields):
  for field in fields:
    unique_field_names.add(field)
    row[field] = obj.find(field).text.strip() if \
                  obj.find(field) is not None else ''
  return row

In [65]:
rows = []
for filename in filenames:
  with open(filename) as f:
    xml_string = f.read()
    root = ET.fromstring(xml_string)

    rbs = root.findall('.//Rb-27s') or root.findall('.//Rb-28s')
    for rb in rbs:
      row_dict = {'file': filename}

      unit = rb.find('.//Jednostka')
      fields = [child.tag for child in unit]
      row_dict = add_fields_to_dict(unit, row_dict, fields)
      for pos in rb.findall('.//Pozycja'):
        fields = [child.tag for child in pos]
        row_dict = add_fields_to_dict(pos, row_dict, fields)
        rows.append(row_dict.copy())


In [66]:
print(rows)
print(len(rows))
print(unique_field_names)

[{'file': 'Sprawozdania[2022][IVKwartał] Wydatki.xml', 'Nazwa': 'Szkoła Podstawowa Nr 7', 'Typ': 'JB', 'Regon': '001070630', 'WK': '10', 'PK': '02', 'GK': '01', 'GT': '1', 'PT': '0', 'Dzial': '801', 'Rozdzial': '80101', 'Grupa': '1300', 'Paragraf': '302', 'P4': '0', 'PL': '10280.00', 'ZA': '9803.89', 'WW': '9803.89'}, {'file': 'Sprawozdania[2022][IVKwartał] Wydatki.xml', 'Nazwa': 'Szkoła Podstawowa Nr 7', 'Typ': 'JB', 'Regon': '001070630', 'WK': '10', 'PK': '02', 'GK': '01', 'GT': '1', 'PT': '0', 'Dzial': '801', 'Rozdzial': '80101', 'Grupa': '1400', 'Paragraf': '401', 'P4': '0', 'PL': '674540.00', 'ZA': '673221.42', 'WW': '658989.84', 'ZO': '14231.58'}, {'file': 'Sprawozdania[2022][IVKwartał] Wydatki.xml', 'Nazwa': 'Szkoła Podstawowa Nr 7', 'Typ': 'JB', 'Regon': '001070630', 'WK': '10', 'PK': '02', 'GK': '01', 'GT': '1', 'PT': '0', 'Dzial': '801', 'Rozdzial': '80101', 'Grupa': '1400', 'Paragraf': '404', 'P4': '0', 'PL': '48295.00', 'ZA': '48276.23', 'WW': '48276.23', 'ZO': '48701.67'},

In [67]:
for filename in filenames:
  output_filename = filename.replace('xml', 'csv')

  with open(output_filename, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=unique_field_names)

      # Write the header row
      writer.writeheader()

      # Write the data rows, filling in empty values for missing fields

      for row in (row for row in rows if row['file'] == filename ):
          writer.writerow({field: row.get(field, '') for field in unique_field_names})
